# Le modèle de l'Oregonator

Le modèle de l'Oregonator s'écrit :

\begin{equation} 
\left\{ 
\begin{aligned} 
\partial_t y_1 & = y_2-y_1\\ 
\varepsilon\, \partial_t y_2 &= q\,y_3-y_3\,y_2+y_2(1-y_2)\\
\mu\,  \partial_t y_3 &= -q\,y_3- y_3\,y_2+f\, y_1
\end{aligned} 
\right. 
\end{equation}

On prendra les paramètres suivant :

\begin{equation} 
\epsilon = 10^{-2},\quad \mu =10^{-4}, \quad f=1,\quad q=2.10^{-4}.
\end{equation}

In [ ]:
import numpy as np
import time

from scipy.integrate import solve_ivp

from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.layouts import column

from mylib.model import oregonator_model
import mylib.integration as integration
from mylib.integration import parareal

output_notebook(hide_banner=True)

In [ ]:
def compute_integration_time(tini, tend, yini, fcn, method, nt, rtol, atol, ntimes=100):
        
    t0 = time.time()
    for i in range(ntimes):
        integration.integrate(method, tini, tend, nt, rtol, atol, yini, fcn)
    t1 = time.time()
    time_average = (t1-t0) / ntimes
    
    return time_average

In [ ]:
def plot_sol(tini, tend, nb_sub, sol_ref, sol):

    # plot solution
    fig_sol_y1 = figure(x_range=(tini-0.25, tend+0.25), width=950, height=400, 
                     title="Solution y1 (click on legend to see reference solution)")
    fig_sol_y2 = figure(x_range=(tini-0.25, tend+0.25), width=950, height=400, 
                     title="Solution y2 (click on legend to see reference solution)")
    fig_sol_y3 = figure(x_range=(tini-0.25, tend+0.25), width=950, height=400, 
                     title="Solution y3 (click on legend to see reference solution)")
    # plot ref solution
    for isub in range(nb_sub):
        fig_sol_y1.line(sol_ref.sol_fine[isub].t, sol_ref.sol_fine[isub].y[0], color="grey", line_alpha=0.5)
        fig_sol_y2.line(sol_ref.sol_fine[isub].t, sol_ref.sol_fine[isub].y[1], color="grey", line_alpha=0.5)
        fig_sol_y3.line(sol_ref.sol_fine[isub].t, sol_ref.sol_fine[isub].y[2], color="grey", line_alpha=0.5)
        
    # plot Y
    fig_sol_y1.x(sol.T, sol.Y[0], line_width=2, size=10)    
    fig_sol_y2.x(sol.T, sol.Y[1], line_width=2, size=10)
    fig_sol_y3.x(sol.T, sol.Y[2], line_width=2, size=10)
    
    # plot fine solution
    for isub in range(nb_sub):
        fig_sol_y1.line(sol.sol_fine[isub].t, sol.sol_fine[isub].y[0])
        fig_sol_y2.line(sol.sol_fine[isub].t, sol.sol_fine[isub].y[1])
        fig_sol_y3.line(sol.sol_fine[isub].t, sol.sol_fine[isub].y[2])

    show(column(fig_sol_y1, fig_sol_y2, fig_sol_y3))

## Solution obtenue par l'algorithme pararéel 

In [ ]:
def parareal_sol():
    
    eps = 1.e-2
    mu  = 1.e-5
    f   = 1.
    q   = 2.e-4
    om = oregonator_model(eps=eps, mu=mu, f=f, q=q)
    fcn = om.fcn
 
    tini = 0. 
    tend = 30.

    yini = (0.5, 1.e-3, (f*0.5)/(q+1.e-3))
    
    scale_factor = np.array([0.3, 1.0, 600.])
        
    # parareal parameters
    nb_sub = 30
    coarse_method = "radau5"
    nc = 100
    rtolc = 1.e-3
    atolc = 1.e-6
    fine_method = "radau5"
    nf = 100
    rtolf = 1.e-8
    atolf = 1.e-8
    max_iter = nb_sub
    eps = 1.e-3
    
    # compute time for coarse integration
    Tg = compute_integration_time(tini, tend, yini, fcn, coarse_method, nb_sub*nc, rtolc, atolf, ntimes=10)
    print(f"Elapsed time (s) for coarse integration : {Tg:.8f}")
    # compute time for fine integration
    Tf = compute_integration_time(tini, tend, yini, fcn, fine_method, nb_sub*nf, rtolf, atolf, ntimes=10)
    print(f"Elapsed time (s) for fine integration   : {Tf:.8f}")
    
    # compute reference solution
    print("\nCompute reference solution ...")
    sol_ref = integration.compute_ref_sol(tini, tend, nb_sub, yini, fcn, 'Radau')
        
    # parareal algorithm
    print("\nParareal algorithm\n")     
    sol = parareal(tini, tend, nb_sub, yini, fcn, coarse_method, nc, rtolc, atolc, fine_method, nf, rtolf, atolf, 
                   max_iter, eps, scale_factor)
    
    print("\nGlobal error:")     
    err = np.zeros(3)
    for ieq in range(3):
        err[ieq] = np.linalg.norm((sol.Y[ieq]-sol_ref.Y[ieq])/scale_factor[ieq])/np.sqrt(nb_sub)
        print(f" {ieq+1}th variable : || Yi - Yref || = {err[ieq]:7.2e} (error between parareal and ref solution)")
        
    print("\nEfficiency :")
    print(f"  nb iteration (nit) = {sol.nit}")
    print(f"  nit/n = {sol.nit/nb_sub}")
    print(f"  Tg/Tf = {Tg/Tf}")
    Tpar = (sol.nit/nb_sub) + (sol.nit+1)*(Tg/Tf) 
    print(f"  Tseq/Tpar = {1/Tpar}")

    # plot solution
    plot_sol(tini, tend, nb_sub, sol_ref, sol)
        
parareal_sol()